In [1]:
!git clone https://github.com/theadityakr/Deblur-Images-using-GAN.git
import os
os.chdir("Deblur-Images-using-GAN/")
%tensorflow_version 1.x
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
!pwd

fatal: destination path 'Deblur-Images-using-GAN' already exists and is not an empty directory.
TensorFlow 1.x selected.
/content/Deblur-Images-using-GAN


#### Importing Libraries

In [2]:
import numpy as np
import glob, cv2, os
import datetime
import tqdm

!pip install h5py==2.10.0

#from deblurgan.utils import write_log
from losses import wasserstein_loss, perceptual_loss
from model import generator_model, discriminator_model, generator_containing_discriminator_multiple_outputs

from keras.callbacks import TensorBoard
from keras.optimizers import Adam
#for saving weights
BASE_DIR = 'weights/'



Using TensorFlow backend.


In [3]:
# Read images from input directory
def load_images(images_path):
    X_images = []
    for img in images_path:
      X_images.append(cv2.resize(cv2.imread(img), input_shape[:-1]))
    X_images = np.array(X_images)
    #Preprocessing i.e normalizing data
    X_images = (X_images - 127.5) / 127.5
    return X_images

# Saving weights after each iteration
def save_all_weights(d, g, epoch_number, current_loss):
    now = datetime.datetime.now()
    save_dir = os.path.join(BASE_DIR, '{}{}'.format(now.month, now.day))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    g.save_weights(os.path.join(save_dir, 'generator_{}_{}.h5'.format(epoch_number, current_loss)), True)
    d.save_weights(os.path.join(save_dir, 'discriminator_{}.h5'.format(epoch_number)), True)

# Training of model
def train_multiple_outputs(images_path, batch_size, epoch_num, critic_updates=5):
    x_path = glob.glob(images_path + "/blurred/*")
    y_path = glob.glob(images_path + "/unblurred/*")
    x_path.sort(); y_path.sort();
    x_train = load_images(x_path)
    y_train = load_images(y_path)

    g = generator_model(input_shape)
    d = discriminator_model(input_shape)
    d_on_g = generator_containing_discriminator_multiple_outputs(g, d, input_shape)

    d_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    d_on_g_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    d.trainable = True
    d.compile(optimizer=d_opt, loss=wasserstein_loss)
    d.trainable = False
    loss = [perceptual_loss, wasserstein_loss]
    loss_weights = [100, 1]
    d_on_g.compile(optimizer=d_on_g_opt, loss=loss, loss_weights=loss_weights)
    d.trainable = True

    output_true_batch, output_false_batch = np.ones((batch_size, 1)), -np.ones((batch_size, 1))

    tensorboard_callback = TensorBoard("../")
    print("returning")

    for epoch in tqdm.tqdm(range(epoch_num)):
        permutated_indexes = np.random.permutation(x_train.shape[0])

        d_losses = []
        d_on_g_losses = []
        for index in range(int(x_train.shape[0] / batch_size)):
            batch_indexes = permutated_indexes[index*batch_size:(index+1)*batch_size]
            image_blur_batch = x_train[batch_indexes]
            image_full_batch = y_train[batch_indexes]

            generated_images = g.predict(x=image_blur_batch, batch_size=batch_size)

            for _ in range(critic_updates):
                d_loss_real = d.train_on_batch(image_full_batch, output_true_batch)
                d_loss_fake = d.train_on_batch(generated_images, output_false_batch)
                d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
                d_losses.append(d_loss)

            d.trainable = False

            d_on_g_loss = d_on_g.train_on_batch(image_blur_batch, [image_full_batch, output_true_batch])
            d_on_g_losses.append(d_on_g_loss)

            d.trainable = True

        # write_log(tensorboard_callback, ['g_loss', 'd_on_g_loss'], [np.mean(d_losses), np.mean(d_on_g_losses)], epoch_num)
        print(np.mean(d_losses), np.mean(d_on_g_losses))
        with open('log.txt', 'a+') as f:
            f.write('{} - {} - {}\n'.format(epoch, np.mean(d_losses), np.mean(d_on_g_losses)))

        save_all_weights(d, g, epoch, int(np.mean(d_on_g_losses)))
    return


#### Set Parameters accordingly

In [4]:
# Setting parameters for our model
input_shape = (256,256,3)
batch_size = 1
epochs = 1
input_directory = "train_data"

#### Training of model with the parameters provided above

In [5]:
train_multiple_outputs(input_directory, batch_size, epochs)

58892288/58889256 [==============================] - 3s 0us/step
returning


  0%|          | 0/1 [00:00<?, ?it/s]

-0.4339799720854823 1764.1526


100%|██████████| 1/1 [11:30<00:00, 690.89s/it]


# Testing

In [6]:
from model import generator_model

In [7]:
# Function to read images from input directory
def load_images(images_path):
    X_images = []
    for img in images_path:
      X_images.append(cv2.resize(cv2.imread(img), input_shape[:-1]))
    X_images = np.array(X_images)
    #Preprocessing i.e normalizing data
    X_images = (X_images - 127.5) / 127.5        
    return X_images

#Creating an instant of model from weight file provided
def test(input_dir, model, output_dir):
    images_path = glob.glob(input_dir + "/*")
    data = load_images(images_path)
    g = generator_model(input_shape)
    g.load_weights(model)
    generated_images = g.predict(data, batch_size=batch_size)
    #deprocessing i.e. un normalize
    generated_images = (generated_images * 127.5 + 127.5).astype('uint8')
    data = (data * 127.5 + 127.5).astype('uint8')
    for i in range(generated_images.shape[0]):
        output = np.concatenate((data[i,:,:,:],generated_images[i, :, :, :]), axis=1)
        img_name = output_dir + "/" + os.path.basename(images_path[i])
        cv2.imwrite(img_name,output)


In [8]:
path = "/content/output"
if not os.path.exists(path):
        os.makedirs(path)

# Setting parameters for our model
input_shape = (256,256,3)
batch_size = 1
input_directory = "test_data/blurred"
model_weights = "generator.h5"
output_directory = "/content/output"

In [9]:
test(input_directory, model_weights, output_directory)